In [ ]:
# stdlib
from os import environ as env

# syft absolute
import syft as sy
from syft.util.test_helpers.email_helpers import get_email_server

In [ ]:
# stdlib
environment = env.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = env.get("CLUSTER_HTTP_PORT_HIGH", "9081")
low_port = env.get("CLUSTER_HTTP_PORT_LOW", "9083")
print(environment, high_port, low_port)

In [ ]:
# when in k8s these are the default values
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
email_server, smtp_server = get_email_server(reset=True)

In [ ]:
root_client_low = sy.login(
    url=f"http://localhost:{low_port}", email=ROOT_EMAIL, password=ROOT_PASSWORD
)
root_client_high = sy.login(
    url=f"http://localhost:{high_port}", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

# Create new admin client

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [ ]:
# create admin account on low side
root_client_low.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client_low.users.search(email=ADMIN_EMAIL)[0].id
root_client_low.users.update(uid=new_user_id, role="admin")

In [ ]:
# create admin account on high side
root_client_high.register(
    name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW
)
# update role
new_user_id = root_client_high.users.search(email=ADMIN_EMAIL)[0].id
root_client_high.users.update(uid=new_user_id, role="admin")

# We cannot delete the root client

In [ ]:
low_client = sy.login(
    url=f"http://localhost:{low_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
root_admin_id = low_client.users.search(email=ROOT_EMAIL)[0].id

In [ ]:
with sy.raises(sy.SyftException):
    low_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [ ]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client_low.register(
    name="x", email=user_email, password=user_pw, password_verify=user_pw
)
## update role
new_user_id2 = root_client_low.users.search(email=user_email)[0].id
root_client_low.users.update(uid=new_user_id2, role="admin")

# delete
root_client_low.users.delete(new_user_id2)

# Cleanup

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()
smtp_server.stop()